## Zad. 1

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
# wczytanie danych
san = pd.read_csv('dane/san.csv', index_col=0, skiprows=2)
san.index = pd.to_datetime(san.index, format='%Y-%m-%d')
san = san.iloc[:, 0]

In [3]:
# procentowy zanualizowany zwrot
annual_return = (1 + san.pct_change().dropna()).prod() ** (252 / (len(san)-1))-1

In [4]:
# ostatnia znana cena akcji
round(san.iloc[-1], 3)

6.85

In [5]:
# premia do opcji sprzedaży
S = 6.85
K = S
T = 0.25
sigma = san.pct_change().dropna().std() * np.sqrt(252)
R = 0.03
n = 10**4  # liczba akcji

In [6]:
# cena Blacka-Scholesa na opcję sprzedaży
def black_scholes(s=S, k=K, t=T, sig=sigma, r=R):
    d1 = (-np.log(s/k) - (r - sig**2/2) * t) / (sig*np.sqrt(t))
    d2 = (-np.log(s/k) - (r + sig**2/2) * t) / (sig*np.sqrt(t))
    return k * np.exp(-r * t) * norm.cdf(d1) - s * norm.cdf(d2)


round(black_scholes(), 2)

0.52

In [7]:
premium = n * round(black_scholes(), 2)
premium

5200.0

Santander Polska posiada 10 000 akcji Banco Santander S.A. (wartość: 68 500 EUR). W celu zabezpieczenia przed ryzykiem spadku cen akcji, możliwe są trzy strategie:
1. Podjęcie ryzyka.
2. Zakup kontraktu forward na sprzedaż akcji z terminem wykonania 3 miesiące. Cena wykonania kontaktu wynosi 6.90 EUR/akcja.
3. Zakup opcji sprzedaży na instrument podstawowy z ceną wykonania 6.85 EUR/akcja, terminem wykonania za 3 miesiące i ceną wykonania (premią) 0.52 EUR/akcja.

In [8]:
def option3(f_vals, strike, premium):
    res = []
    executed = []
    for val in f_vals:
        payoff = max(strike - val, 0)
        net = (val + payoff) * n - premium 
        res.append(net)
        executed.append("T" if payoff > 0 else "F")
    return res, executed

In [9]:
future_vals = np.array([6.6, 6.7, 6.8, 6.85, 6.9, 7., 7.1])
result = pd.DataFrame({
'Wartość kontraktu': future_vals * n,
'Efekt strategii 1 – podjęcie ryzyka': future_vals * n,
'Efekt strategii 2 - sprzedaż kontraktu forward': [6.9 * n] * 7,
'Efekt strategii 3 - zakup opcji sprzedaży': option3(future_vals, K, premium)[0],
'Wykonanie opcji sprzedaży': option3(future_vals, K, premium)[1]},
    index = future_vals)
result

,Wartość kontraktu,Efekt strategii 1 – podjęcie ryzyka,Efekt strategii 2 - sprzedaż kontraktu forward,Efekt strategii 3 - zakup opcji sprzedaży,Wykonanie opcji sprzedaży
6.60,66000.0,66000.0,69000.0,63300.0,T
6.70,67000.0,67000.0,69000.0,63300.0,T
6.80,68000.0,68000.0,69000.0,63300.0,T
6.85,68500.0,68500.0,69000.0,63300.0,F
6.90,69000.0,69000.0,69000.0,63800.0,F
7.00,70000.0,70000.0,69000.0,64800.0,F
7.10,71000.0,71000.0,69000.0,65800.0,F
